In [95]:
import pandas as pd
train_df = pd.read_csv('../../Structured/Walmart-Amazon/merged_train.csv')
train_pos = train_df[train_df.label==1].copy()
train_neg = train_df[train_df.label==0].copy()

In [96]:
import random
import string
def randomStringDigits(stringLength=6):
    """Generate a random string of letters and digits """
    lettersAndDigits = string.ascii_letters + string.digits
    return ''.join(random.choice(lettersAndDigits) for i in range(stringLength))

In [97]:
train_pos['ltable_modelno'] = train_pos['ltable_modelno'].fillna(train_pos['rtable_modelno'])

In [98]:
train_pos['rtable_modelno'] = train_pos['rtable_modelno'].fillna(train_pos['ltable_modelno'])
train_pos.head(10)

,label,ltable_title,ltable_category,ltable_brand,ltable_modelno,ltable_price,rtable_title,rtable_category,rtable_brand,rtable_modelno,rtable_price,id
8,1,elite screens cinegray prime vision series fix...,electronics - general,elite screens,pvr135h1,1069.00,cinegray prime vision series fixed frame scree...,projection screens,elite,pvr135h1,1069.00,4699
35,1,michigan wolverines iphone 4 case silicone cover,electronics - general,tribeca,fva3437,24.99,michigan wolverines iphone 4 case silicone cover,computers accessories,tribeca,fva3437,17.99,1353
49,1,innovera d2130b black compatible high-yield to...,printers,innovera,d2130b,58.90,new-innovera d2130b - d2130b compatible high-y...,computer accessories,innovera,d2130b,78.00,1564
366,1,da-lite da-glas standard rear projection scree...,electronics - general,da-lite,27639,4448.99,da-lite 27639 da-glas standard rear projection...,projection screens,da-lite,27639,NaN,5013
510,1,pc treasures wireless optical mouse 2.4 ghz pu...,mice,pc treasures,07227,17.82,pc treasures mighty mini mouse - 2.4 ghz wireless,mice,pc treasures,07227,14.69,4586
536,1,sumdex slr camera sling pack,mp3 accessories,sumdex,144952,39.99,slr camera sling pack,cases bags,sumdex,poc-484bk,44.59,5798
537,1,xerox 108r00645 imaging unit for phaser 6300 a...,printers,xerox,108r00645,244.84,xerox printers imaging unit phaser 6300 6350 1...,printer ink toner,xerox,108r00645,253.79,5226
573,1,hp q6511a hp 11a toner 6000 page-yield black,printers,hp,q6511a,191.03,hp q6511a laserjet 11a black print cartridge i...,laser printer toner,hp,q6511a,NaN,2677
577,1,hp 56 black inkjet cartridge c6656an,printers,hp,c6656an,23.97,genuine hp 56 c6656an black inkjet print cartr...,inkjet printer ink,hp,c6656an,24.19,435
598,1,toshiba ph3100u-1e3s 1tb usb 3.0 desktop hard ...,hard drives,toshiba,ph3100u-1e3s,89.00,toshiba 1 tb usb 3.0 external hard drive ph310...,external hard drives,toshiba,ph3100u-1e3s,80.23,1329


In [99]:
train_neg.ltable_modelno.isna().sum(),train_neg.rtable_modelno.isna().sum()

(180, 1873)

In [100]:
train_neg.loc[train_neg['ltable_modelno'].isna(), 'ltable_modelno'] = train_neg.loc[train_neg['ltable_modelno'].isna(), 'ltable_modelno'].apply(lambda x: randomStringDigits(10))

In [101]:
train_neg.loc[train_neg['rtable_modelno'].isna(), 'rtable_modelno'] = train_neg.loc[train_neg['rtable_modelno'].isna(), 'rtable_modelno'].apply(lambda x: randomStringDigits(10))

In [102]:
train_neg.ltable_modelno.isna().sum(),train_neg.rtable_modelno.isna().sum()

(0, 0)

In [103]:
train_neg.head()

,label,ltable_title,ltable_category,ltable_brand,ltable_modelno,ltable_price,rtable_title,rtable_category,rtable_brand,rtable_modelno,rtable_price,id
0,0,elite screens cinewhite cinema235 series fixed...,electronics - general,elite screens,r85wh1-wide,409.0,cinegray ezframe series fixed frame screen - 1...,projection screens,elite,JGMp6dReW1,879.0,5583
1,0,elite screens cinewhite cinema235 series fixed...,electronics - general,elite screens,r85wh1-wide,409.0,powergain prime vision series fixed frame scre...,projection screens,elite,ggAwX7K1pp,1069.0,1572
2,0,elite screens cinegray prime vision series fix...,electronics - general,elite screens,pvr135h1,1069.0,powergain prime vision series fixed frame scre...,projection screens,elite,UIy5npC5UK,1069.0,110
3,0,elite screens cinewhite cinema235 series fixed...,electronics - general,elite screens,r85wh1-wide,409.0,cinewhite prime vision series fixed frame scre...,projection screens,elite,gPFzikLEHz,929.0,1938
4,0,elite screens cinegray prime vision series fix...,electronics - general,elite screens,pvr135h1,1069.0,cinewhite prime vision series fixed frame scre...,projection screens,elite,t6dsP2jCfF,929.0,5694


In [104]:
train_df_without_null = pd.concat([train_neg,train_pos])
train_df_without_null.to_csv('../../Structured/Walmart-Amazon/train_without_nullcodes.csv',index=False)

## Train model with new dataset

In [106]:
import deepmatcher as dm
train,validation,test = dm.data.process('../../Structured/Walmart-Amazon/',train='train_without_nullcodes.csv',
                       validation='merged_valid.csv',test='merged_test.csv',left_prefix='ltable_',
                       right_prefix='rtable_',cache='notnull.pth')

In [107]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.run_train(train,validation,best_save_path='../../models/walmartamazon_hybrid_nonull.pth',epochs=20,
                       pos_neg_ratio=10,batch_size=16)

* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:54


Finished Epoch 1 || Run Time:   50.6 | Load Time:    4.8 || F1:  32.37 | Prec:  19.68 | Rec:  90.97 || Ex/s: 110.94

===>  EVAL Epoch 1


0% [█████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    6.2 | Load Time:    1.5 || F1:  49.01 | Prec:  42.69 | Rec:  57.51 || Ex/s: 268.38

* Best F1: 49.00662251655629
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:56


Finished Epoch 20 || Run Time:   52.7 | Load Time:    4.8 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 106.81

===>  EVAL Epoch 20


0% [█████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 20 || Run Time:    6.6 | Load Time:    1.5 || F1:  60.61 | Prec:  64.71 | Rec:  56.99 || Ex/s: 251.18

---------------------

Loading best model...
Training done.


60.60606060606061

In [108]:
hybrid_model.run_eval(test)

===>  EVAL Epoch 19


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 19 || Run Time:    3.5 | Load Time:    1.5 || F1:  59.56 | Prec:  63.01 | Rec:  56.48 || Ex/s: 411.32



59.56284153005465